# 使用深度学习技术在未来的轻子对撞机中寻找长寿命粒子

## 概述
在寻找超出标准模型（BSM）的物理现象时，长寿命粒子（LLPs）提供了一个明确的信号。它们具有独特的探测器特征，衰变长度对应于纳秒或更长的寿命。电子对撞机允许在干净的环境中进行LLPs搜索，并且当结合深度学习（DL）技术时，这些搜索可以发挥其全部物理潜力。

本教程介绍了论文中异构图神经网络的研究背景和技术路径，并展示了如何通过MindSpore训练和快速推断模型。 更多信息可以在以下位置找到：[论文](https://arxiv.org/abs/2401.05094)。

## 技术路径
解决这个问题的具体流程如下：

创建数据集

模型构建

损失函数

模型训练

# 异构图神经网络
论文使用异构图表征电子对撞机上的正负电子到希格斯玻色子和Z玻色子事例，将电磁量能器和径迹探测器上的探测信号分别表示为图上的两种异构节点和边，通过信息传递机制更新节点和边的表示，通过共享权重进行异构图的信息交换，实现事例的分类。

下图为网络架构：

![LorentzNet](./images/LLP.PNG)

In [1]:
import mindspore
from mindspore import nn, ops, Tensor
from mindspore.train import Model
import mindspore.numpy as mnp
import numpy as np
from mindspore.common.initializer import XavierUniform
from mindspore import context
context.set_context(mode=1, device_target="CPU")
from src.model import HeteroNet

[WARNING] ME(1153772,7fa36fda0740,python):2024-12-04-16:51:39.091.835 [mindspore/ccsrc/runtime/hardware/device_context_manager.cc:564] SelectGpuPlugin] Env CUDA_HOME is /usr/local/cuda, but can not find suitable gpu plugin.


In [2]:
model = HeteroNet(embed_size=32, clu_node_dim=6, clu_edge_dim=9, trk_node_dim=5, trk_edge_dim=5)

In [3]:
import dgl
graphs = dgl.load_graphs('data/sig/sig_2jvv_m50GeV_t1_1_100_heter.bin')
g = graphs[0]
l = graphs[1]['labels'].numpy()

In [4]:
import mindspore.dataset as mds
ds = mds.NumpySlicesDataset((l, g), column_names=['label', 'graph'], sampler=mds.RandomSampler())

In [5]:
        clu_ndata = Tensor(g[0].nodes['cluster'].data['scalars'].numpy(), mindspore.float32)
        print(clu_ndata.shape)
        clu_edata = Tensor(g[0].edges['clu_inter'].data['interact'].numpy(), mindspore.float32)
        print(clu_edata.shape)
        clu_src, clu_dst = g[0].edges(etype='clu_inter')
        clu_src = Tensor(clu_src.numpy())
        clu_dst = Tensor(clu_dst.numpy())
        print(clu_src.shape)
        trk_ndata = Tensor(g[0].nodes['tracker'].data['scalars'].numpy(), mindspore.float32)
        print(trk_ndata.shape)
        trk_edata = Tensor(g[0].edges['trk_inter'].data['interact'].numpy(), mindspore.float32)
        print(trk_edata.shape)
        trk_src, trk_dst = g[0].edges(etype='trk_inter')
        trk_src = Tensor(trk_src.numpy())
        trk_dst = Tensor(trk_dst.numpy())
        print(trk_src.shape, type(trk_src[0]), trk_src[0])

(4, 6)
(12, 9)
(12,)
(10, 5)
(90, 5)
(90,) <class 'mindspore.common._stub_tensor.StubTensor'> 0


In [6]:
model(clu_ndata, clu_edata, clu_src, clu_dst, trk_ndata, trk_edata, trk_src, trk_dst)

(4, 32) (12, 32) (10, 32) (90, 32)
(12, 32) (12, 32) (12, 32)
(12, 96)
(4, 32) (12, 32)
(90, 32) (90, 32) (90, 32)
(90, 96)
(10, 32) (90, 32)
h (14, 32)
e (102, 32)
h (14, 32)
e (102, 32)
hclu (4, 32)
htrk (10, 32)
eclu (12, 32)
etrk (90, 32)
(12, 32) (12, 32) (12, 32)
(12, 96)
(4, 32) (12, 32)
(90, 32) (90, 32) (90, 32)
(90, 96)
(10, 32) (90, 32)
h (14, 32)
e (102, 32)
h (14, 32)
e (102, 32)
hclu (4, 32)
htrk (10, 32)
eclu (12, 32)
etrk (90, 32)
(12, 32) (12, 32) (12, 32)
(12, 96)
(4, 32) (12, 32)
(90, 32) (90, 32) (90, 32)
(90, 96)
(10, 32) (90, 32)
h (14, 32)
e (102, 32)
h (14, 32)
e (102, 32)
hclu (4, 32)
htrk (10, 32)
eclu (12, 32)
etrk (90, 32)
(12, 32) (12, 32) (12, 32)
(12, 96)
(4, 32) (12, 32)
(90, 32) (90, 32) (90, 32)
(90, 96)
(10, 32) (90, 32)
h (14, 32)
e (102, 32)
h (14, 32)
e (102, 32)
hclu (4, 32)
htrk (10, 32)
eclu (12, 32)
etrk (90, 32)
(12, 32) (12, 32) (12, 32)
(12, 96)
(4, 32) (12, 32)
(90, 32) (90, 32) (90, 32)
(90, 96)
(10, 32) (90, 32)
h (14, 32)
e (102, 32)
h (

Tensor(shape=[1, 5], dtype=Float32, value=
[[ 1.77858174e-01, -2.18985915e-01,  2.13854387e-01, -2.72543699e-01,  8.47831070e-02]])